# Symbolic Rules Deep Dive

This notebook explores:
1. The fluent RuleBuilder API
2. Rule validation
3. Forward chaining in detail
4. Backward chaining (SLD prover)
5. Betti number analysis

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))

from nesy.core.types import Predicate, SymbolicRule
from nesy.symbolic.rules import RuleBuilder, RuleValidator
from nesy.symbolic.engine import SymbolicEngine
from nesy.symbolic.prover import BackwardChainer
from nesy.symbolic.betti import BettiAnalyser

## 1. Fluent Rule Builder

Chain `.if_fact()`, `.then()`, `.with_weight()`, `.in_domain()` for readable rule construction.

In [ ]:
rule1 = (
    RuleBuilder('fever_flu')
    .if_fact('fever')
    .if_fact('cough')
    .then('flu_likely')
    .with_weight(0.85)
    .in_domain('medical')
    .description('Fever + cough suggests flu')
    .build()
)

rule2 = (
    RuleBuilder('flu_rest')
    .if_fact('flu_likely')
    .then('recommend_rest')
    .with_weight(0.90)
    .build()
)

rule3 = (
    RuleBuilder('flu_hydration')
    .if_fact('flu_likely')
    .then('recommend_hydration')
    .with_weight(0.88)
    .build()
)

rules = [rule1, rule2, rule3]
for r in rules:
    print(f'{r.id}: w={r.weight}')

## 2. Rule Validation

In [ ]:
validator = RuleValidator()
errors = validator.validate(rules)
if errors:
    for e in errors:
        print(f'  Error: {e}')
else:
    print('All rules valid!')

## 3. Forward Chaining

In [ ]:
engine = SymbolicEngine(domain='medical')
engine.load_rules(rules)

facts = {Predicate('fever'), Predicate('cough')}
derived, steps, confidence = engine.reason(facts)

print(f'Derived: {[str(d) for d in derived]}')
print(f'Confidence: {confidence:.3f}')
for step in steps:
    print(f'  Step {step.step_number}: {step.description}')

## 4. Backward Chaining (SLD Prover)

In [ ]:
prover = BackwardChainer(rules=rules, max_depth=20)
goal = Predicate('recommend_rest')
result = prover.prove(goal, facts)

print(f'Goal: {goal}')
print(f'Proved: {result.proved}')
print(f'Depth: {result.depth_reached}')
print(f'Rules used: {result.rules_used}')
print(f'Confidence: {result.confidence:.3f}')
print(result.explain())

## 5. Betti Number Analysis

In [ ]:
all_preds = list(derived)
beta_0 = BettiAnalyser.compute(all_preds)
coherence = BettiAnalyser.coherence_score(all_preds)
diagnostic = BettiAnalyser.diagnose(all_preds)

print(f'Beta_0: {beta_0}')
print(f'Coherence: {coherence:.3f}')
print(f'Diagnostic: {diagnostic}')

## 6. Anchored Rules

In [ ]:
anchor_rule = (
    RuleBuilder('critical_safety')
    .if_fact('chest_pain')
    .if_fact('shortness_of_breath')
    .then('cardiac_emergency')
    .with_weight(0.99)
    .as_anchor()
    .build()
)
print(f'Rule: {anchor_rule.id}, Immutable: {anchor_rule.immutable}')